In [1]:
import pandas as pd
import geopandas as gpd

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [3]:
df = pd.read_csv('resultados/modelo_preditivo.csv.zip')

/tmp/ipykernel_124476/1623530861.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo_preditivo.csv.zip')
